# PDX: Helper functions to run SQL on Pandas DataFrames

Leverages DuckDB, which has some nice SQL language extensions:

- https://duckdb.org/2022/05/04/friendlier-sql.html
- https://duckdb.org/2022/11/14/announcing-duckdb-060.html

In [1]:
import pdx
print(pdx.__version__)

0.4.0


In [2]:
iris = pdx.data.get_iris()
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [3]:
iris.sql?

Signature: iris.sql(s='', tbl_name='_df')
Docstring:
Run a DuckDB SQL query against Pandas DataFrame.

DataFrame will be referred to by string given in `tbl_name`.
The query starts with a `from {table_name}` clause.
Since DuckDB 0.6, the `select` clause is optional, defaulting to `select *`.

Examples
--------

```
df.sql('select col_a where col_b > 0')
```

is equivalent to the query

```
select
    col_a
from
    _df
where
    col_b > 0
```


`df.sql('where col_b > 0')` is equivalent to

```
select
    *
from
    _df
where
    col_b > 0
```

Note that `df.sql()` is equivalent to

```
select * from _df
```
File:      ~/work/pdx/src/pdx/_pandas.py
Type:      method


In [4]:
iris.sql('select *')

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [5]:
iris.sql('')

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [6]:
iris.sql()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [7]:
iris.sql("""
select
    species,
    count(*)
        as num,
group by
    1
""")

,species,num
0,setosa,50
1,versicolor,50
2,virginica,50


Equivalently, can also run the function and specify the DataFrame/table explicitly.

In [8]:
pdx.sql("""
select
    species,
    count(*)
        as num,
from
    tbl
group by
    1
""", tbl=iris)

,species,num
0,setosa,50
1,versicolor,50
2,virginica,50


Or, using the usual default table name, `_df`.

In [9]:
pdx.sql("""
select
    species,
    count(*)
        as num,
from
    _df
group by
    1
""", _df=iris)

,species,num
0,setosa,50
1,versicolor,50
2,virginica,50


# Joining multiple tables/dataframes

In [10]:
s = """
select
    species,
    avg(petal_width)
        as avg_petal_width,
group by
    1
"""

df2 = iris.sql(s)

df2

,species,avg_petal_width
0,setosa,0.246
1,versicolor,1.326
2,virginica,2.026


In [11]:
s = """
select
      iris.*
    , df2.avg_petal_width
        as species_avg_petal_width
from
    iris
left join
    df2
on
    iris.species = df2.species
"""

pdx.sql(s, iris=iris, df2=df2)

,sepal_length,sepal_width,petal_length,petal_width,species,species_avg_petal_width
0,5.1,3.5,1.4,0.2,setosa,0.246
1,4.9,3.0,1.4,0.2,setosa,0.246
2,4.7,3.2,1.3,0.2,setosa,0.246
3,4.6,3.1,1.5,0.2,setosa,0.246
4,5.0,3.6,1.4,0.2,setosa,0.246
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,2.026
146,6.3,2.5,5.0,1.9,virginica,2.026
147,6.5,3.0,5.2,2.0,virginica,2.026
148,6.2,3.4,5.4,2.3,virginica,2.026


## Alternatively, using a CTE

When a query string starts with "`with`", `pdx` won't prepend `from {tbl_name}` to make working with CTEs (based on a single DataFrame) easier.

In [12]:
s = """
with

  avg as
(
select
    species,
    avg(petal_width)
        as avg_petal_width,
from
    _df
group by
    1
)

select
      d.*
    , a.avg_petal_width
        as species_avg_petal_width
from
    _df as d
left join
    avg as a
on
    d.species = a.species
"""

iris.sql(s)

,sepal_length,sepal_width,petal_length,petal_width,species,species_avg_petal_width
0,5.1,3.5,1.4,0.2,setosa,0.246
1,4.9,3.0,1.4,0.2,setosa,0.246
2,4.7,3.2,1.3,0.2,setosa,0.246
3,4.6,3.1,1.5,0.2,setosa,0.246
4,5.0,3.6,1.4,0.2,setosa,0.246
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,2.026
146,6.3,2.5,5.0,1.9,virginica,2.026
147,6.5,3.0,5.2,2.0,virginica,2.026
148,6.2,3.4,5.4,2.3,virginica,2.026


Using `using` syntax:

In [13]:
s = """
with

  avg as
(
select
    species,
    avg(petal_width)
        as species_avg_petal_width,
from
    _df
group by
    1
)

select
    *
from
    _df
left join
    avg
using
    (species)
"""

iris.sql(s)

,sepal_length,sepal_width,petal_length,petal_width,species,species_avg_petal_width
0,5.1,3.5,1.4,0.2,setosa,0.246
1,4.9,3.0,1.4,0.2,setosa,0.246
2,4.7,3.2,1.3,0.2,setosa,0.246
3,4.6,3.1,1.5,0.2,setosa,0.246
4,5.0,3.6,1.4,0.2,setosa,0.246
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,2.026
146,6.3,2.5,5.0,1.9,virginica,2.026
147,6.5,3.0,5.2,2.0,virginica,2.026
148,6.2,3.4,5.4,2.3,virginica,2.026


# Pandas DataFrame helper functions

A few helper functions for common patterns to extract data from dataframes.

## `df.aslist()`

In [14]:
out = iris.sql('select species group by 1')

In [15]:
out

,species
0,setosa
1,versicolor
2,virginica


In [16]:
out.aslist()

['setosa', 'versicolor', 'virginica']

In [17]:
iris.sql('select species group by 1').aslist()

['setosa', 'versicolor', 'virginica']

## `df.asdict()`

In [18]:
out = iris.sql("""
select
    avg(sepal_length),
    avg(sepal_width),
    avg(petal_length),
    avg(petal_width),
""")

In [19]:
out

,avg(sepal_length),avg(sepal_width),avg(petal_length),avg(petal_width)
0,5.843333,3.057333,3.758,1.199333


In [20]:
out.asdict()

{'avg(sepal_length)': 5.843333333333335,
 'avg(sepal_width)': 3.057333333333334,
 'avg(petal_length)': 3.7580000000000027,
 'avg(petal_width)': 1.199333333333334}

In [21]:
iris.sql("""
select
    avg(sepal_length),
    avg(sepal_width),
    avg(petal_length),
    avg(petal_width),
""").asdict()

{'avg(sepal_length)': 5.843333333333335,
 'avg(sepal_width)': 3.057333333333334,
 'avg(petal_length)': 3.7580000000000027,
 'avg(petal_width)': 1.199333333333334}

## `df.asitem()`

In [22]:
out = iris.sql('select count(*)')
out

,count_star()
0,150


In [23]:
out.asitem()

150

In [24]:
iris.sql('select count(*)').asitem()

150

# PRQL

You can also query with PRQL:

- https://github.com/prql/prql
- https://prql-lang.org/

In [25]:
iris.prql('')

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [26]:
iris.prql("""
filter sepal_length > 6
take 5
""")

,sepal_length,sepal_width,petal_length,petal_width,species
0,7.0,3.2,4.7,1.4,versicolor
1,6.4,3.2,4.5,1.5,versicolor
2,6.9,3.1,4.9,1.5,versicolor
3,6.5,2.8,4.6,1.5,versicolor
4,6.3,3.3,4.7,1.6,versicolor


In [27]:
pdx.prql("""
from tbl
filter sepal_length > 6
take 5
""", tbl=iris)

,sepal_length,sepal_width,petal_length,petal_width,species
0,7.0,3.2,4.7,1.4,versicolor
1,6.4,3.2,4.5,1.5,versicolor
2,6.9,3.1,4.9,1.5,versicolor
3,6.5,2.8,4.6,1.5,versicolor
4,6.3,3.3,4.7,1.6,versicolor


In [28]:
pdx.prql('from tbl', tbl=iris)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [29]:
iris.prql("""
filter sepal_length > 5 and sepal_length < 6
filter sepal_length < 6
derive new_col = sepal_length + sepal_width

group [species] (
  aggregate [
    the_average = average sepal_length,
    the_sum = sum new_col,
  ]
)
""")

,species,the_average,the_sum
0,setosa,5.313636,198.6
1,versicolor,5.604348,191.7
2,virginica,5.766667,51.1
